In [1]:
from minimal_agent import EmulatorClient
from PIL import Image

from anthropic import Anthropic
from rich.console import Console

from morphcloud.api import MorphCloudClient
from dotenv import load_dotenv
import os

from minimal_agent import EmulatorClient
load_dotenv()
console=Console()


python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 6
python-dotenv could not parse statement starting at line 7


In [2]:


# Create the MorphCloud client
morph_client = MorphCloudClient(api_key=os.getenv("MORPH_API_KEY"))

# Start instance from snapshot
console.print("Starting instance from snapshot...")
instance = morph_client.instances.start(
    snapshot_id=os.getenv("SNAPSHOT_ID"), ttl_seconds=60 * 60 * 24  # 24 hours
)

# Wait for instance to be ready
console.print("Waiting for instance to be ready...")
instance.wait_until_ready()

# Get the instance URL
instance_url = next(
    service.url
    for service in instance.networking.http_services
    if service.name == "web"
)

remote_desktop_url = next(
    service.url
    for service in instance.networking.http_services
    if service.name == "novnc"
)

novnc_url = f"{remote_desktop_url}/vnc_lite.html"
console.print(f"Pokemon remote desktop available at: {novnc_url}")


Starting instance from snapshot...

Waiting for instance to be ready...

Pokemon remote desktop available at: https://novnc-morphvm-6yupbncf.http.cloud.morph.so/vnc_lite.html

In [3]:
#initialize client
client = EmulatorClient(instance_url)
console.print(f"Instance url at ",instance_url)

Instance url at  https://web-morphvm-6yupbncf.http.cloud.morph.so

In [5]:
image = client.get_screenshot_base64()

In [4]:
client.get_game_state()

{'collision_map': '+----------+\n|██████████|\n|██████████|\n|··████··██|\n|··█·██··██|\n|····↓·S···|\n|·········█|\n|··········|\n|······████|\n|··········|\n+----------+\n\nLegend:\n█ - Wall/Obstacle\n· - Path/Walkable\nS - Sprite\n↑/↓/←/→ - Player (facing direction)',
 'coordinates': [12, 6],
 'game_state': 'Player: CLAUDE\nRival: WACLAUD\nMoney: $2586\nLocation: ROUTE 4\nCoordinates: (12, 6)\nValid Moves: down, left, right\nBadges: BOULDER\nInventory:\n  POKé BALL x11\n  ANTIDOTE x1\n  POTION x2\n  TM34 x1\nDialog: None\n\nPokemon Party:\n\nN (NIDORAN M):\nLevel 10 - HP: 29/29\nTypes: POISON\n- TACKLE (PP: 35)\n- LEER (PP: 30)\n- HORN ATTACK (PP: 25)\n\nB (PIDGEY):\nLevel 12 - HP: 34/34\nTypes: NORMAL, FLYING\n- GUST (PP: 35)\n- SAND ATTACK (PP: 15)\n- QUICK ATTACK (PP: 30)\n\nS (BULBASAUR):\nLevel 15 - HP: 44/44\nTypes: GRASS, POISON\n- TACKLE (PP: 35)\n- GROWL (PP: 40)\n- LEECH SEED (PP: 10)\n- VINE WHIP (PP: 10)\n\nP (PIKACHU):\nLevel 12 - HP: 33/33\nTypes: ELECTRIC\n- THUNDERSH

In [6]:
client.get_team().split("Pokemon Party")[1]

':\n\nN (NIDORAN M):\nLevel 10 - HP: 29/29\nTypes: POISON\n- TACKLE (PP: 35)\n- LEER (PP: 30)\n- HORN ATTACK (PP: 25)\n\nB (PIDGEY):\nLevel 12 - HP: 34/34\nTypes: NORMAL, FLYING\n- GUST (PP: 35)\n- SAND ATTACK (PP: 15)\n- QUICK ATTACK (PP: 30)\n\nS (BULBASAUR):\nLevel 15 - HP: 44/44\nTypes: GRASS, POISON\n- TACKLE (PP: 35)\n- GROWL (PP: 40)\n- LEECH SEED (PP: 10)\n- VINE WHIP (PP: 10)\n\nP (PIKACHU):\nLevel 12 - HP: 33/33\nTypes: ELECTRIC\n- THUNDERSHOCK (PP: 30)\n- GROWL (PP: 40)\n- THUNDER WAVE (PP: 20)\n'

In [7]:
#stop the instance
instance.stop()

In [ ]:
import cv2
import numpy as np
import base64
from io import BytesIO
from PIL import Image

def decode_base64_image(base64_string):
    """Decodes a base64-encoded image into an OpenCV image."""
    image_data = base64.b64decode(base64_string)
    image = Image.open(BytesIO(image_data))
    return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

def encode_image_to_base64(image):
    """Encodes an OpenCV image to a base64 string."""
    _, buffer = cv2.imencode('.png', image)
    return base64.b64encode(buffer).decode('utf-8')

def detect_features(image_b64):
    """Detects doors, caves, and dark areas in an image and overlays highlights."""
    image = decode_base64_image(image_b64)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Threshold to detect dark areas (caves, shadows, etc.)
    _, dark_mask = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)
    
    # Define color ranges for possible doors (brown/wooden) and caves (dark openings)
    lower_brown = np.array([10, 50, 50], dtype=np.uint8)
    upper_brown = np.array([30, 255, 255], dtype=np.uint8)
    
    lower_dark = np.array([0, 0, 0], dtype=np.uint8)
    upper_dark = np.array([50, 50, 50], dtype=np.uint8)
    
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Masks for doors and caves
    door_mask = cv2.inRange(hsv, lower_brown, upper_brown)
    cave_mask = cv2.inRange(image, lower_dark, upper_dark)
    
    # Find contours for detected regions
    contours_dark, _ = cv2.findContours(dark_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_door, _ = cv2.findContours(door_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_cave, _ = cv2.findContours(cave_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Overlay highlights on the original image
    highlighted_image = image.copy()
    cv2.drawContours(highlighted_image, contours_door, -1, (0, 255, 0), 2)  # Green for doors
    cv2.drawContours(highlighted_image, contours_cave, -1, (255, 0, 0), 2)  # Blue for caves
    cv2.drawContours(highlighted_image, contours_dark, -1, (0, 0, 255), 2)  # Red for dark areas
    
    # Encode the image back to base64
    highlighted_image_b64 = encode_image_to_base64(highlighted_image)
    
    return {
        "doors": len(contours_door),
        "caves": len(contours_cave),
        "dark_areas": len(contours_dark),
        "highlighted_image_b64": highlighted_image_b64
    }



In [12]:
image = client.get_screenshot_base64()
detect_features(image)

{'doors': 0,
 'caves': 87,
 'dark_areas': 87,
 'highlighted_image': 'highlighted_image.png'}